In [1]:
from orl import _findOgr, _findOsgeoPy, _findOsgeoShell

In [2]:
_findOgr()

'C:\\OSGeo4W\\bin\\ogr2ogr.exe'

In [3]:
_findOsgeoPy()

'C:\\OSGeo4W\\bin\\python.exe'

In [4]:
_findOsgeoShell()

'C:\\OSGeo4W\\OSGeo4W.bat'

## Making updates to spatialite dbs

In order to make inserts, deletes on the spatialite database, we need to call the external python included with OSGeo4W to get access to pyspatialite. Below is the template for inserts. 

The only trick is that you have to pass the list of tree_uid's as text. Passing directly as a python object confuses that subprocess module and passes each id as a variable. To get the list back, we use the 'eval' function which evaluates text as code. 

In [1]:
%%writefile UpdateSpatialiteTimestamp.py

import sys
from pyspatialite import dbapi2 as db

def _buildUpdateWhereClauseSpatialite(table, update_field,
                                      lookup_field, valueList):
    valueList = ["'%s'" % value for value in valueList]
    whereClause = (
        "update %s set %s = datetime('now', 'localtime') where %s IN(%s)" % (
            table, update_field, lookup_field, ', '.join(map(str, valueList))))
    return whereClause

database = sys.argv[1]
ids = eval(sys.argv[2])
query = _buildUpdateWhereClauseSpatialite('SuspiciousRecheck', 'pp_timestamp', 'tree_uid', ids)
# print 'the database is {}'.format(database)
# print 'the ids are {}'.format(ids)
# print 'the query is {}'.format(query)

# print 'connecting'
conn = db.connect(database)
# print 'connected'
c = conn.cursor()
# print 'executing'
c.execute(query)
# print 'committing'
conn.commit()
# print 'commited'
conn.close()

Overwriting UpdateSpatialiteTimestamp.py


Example code that takes the suspicious trees updated by Group 18 climbers and get's the tree_uids. Updates those trees in the spatialite database.

In [3]:
%%writefile TestSpatialiteTimestamp.py

import subprocess
import os
import arcpy
from orl import _findOgr, _findOsgeoPy, _findOsgeoShell

test_db = r'J:\Project_OpsData\OH\3735\ClimberResurveyOH.sqlite'

bu = r'I:\ROAM_Data\GDBs\ROAM_Data_Backups\Backup_07062016.gdb'
arcpy.env.workspace = bu
fc = 'Group18_Recheck_SuspiciousUpdates_processed'
tree_ids = (
        [row[0] for row in arcpy.da.SearchCursor(
            fc, ['tree_uid'])])

# print query
# print test_db

command = [_findOsgeoShell(), _findOsgeoPy(), 'UpdateSpatialiteTimestamp.py', test_db, str(tree_ids)]
subprocess.check_call(command, shell =True)

Overwriting TestSpatialiteTimestamp.py


In [5]:
# Testing code
!python TestSpatialiteTimestamp.py

 
OSGEO4W home is C:\OSGeo4W\ 

the database is J:\Project_OpsData\OH\3735\ClimberResurveyOH.sqlite
the ids are [u'{02ae2d1f-0a9f-42ec-99d0-185d7e06ac78}']
the query is update SuspiciousRecheck set pp_timestamp = datetime('now', 'localtime') where tree_uid IN('{02ae2d1f-0a9f-42ec-99d0-185d7e06ac78}')
connecting
connected
executing
committing
commited


In [1]:
import sqlite3

conn = sqlite3.connect(r'J:\Project_OpsData\OH\3735\ClimberResurveyOH.sqlite')
c = conn.cursor()
query = (
    'select final_inspection_date, pp_timestamp from SuspiciousRecheck where '
    'tree_uid = \'{02ae2d1f-0a9f-42ec-99d0-185d7e06ac78}\'')
c.execute(query)
print c.fetchall()
conn.close()

[(u'2016-07-07 09:16:14', u'2016-07-07 09:16:14')]


The timestamp is successfully updated (they are equal b/c of the update trigger on the database. But this is fine because our query for updates looks for 'final_inspection_date' > 'pp_timestamp'

In [8]:
update_spatialite_script = (
    r'\\AAPOHAM3PFFS10\info\ROAM_Data\Roam_Settings/'
    r'DataProcessing\UpdateSpatialiteTimestamp.py')

In [9]:
import os
os.path.exists(update_spatialite_script)

True

In [51]:
tree_ids = [u'{a5bea75b-0a55-42b9-9a5a-7bbb47518b87}', u'{9821961d-c657-44b0-995d-718e90bff932}']
str(tree_ids)

"[u'{a5bea75b-0a55-42b9-9a5a-7bbb47518b87}', u'{9821961d-c657-44b0-995d-718e90bff932}']"

In [1]:
import arcpy
bu = r'I:\ROAM_Data\GDBs\ROAM_Data_Backups\Backup_07062016.gdb'
arcpy.env.workspace = bu
fc = 'Group18_Recheck_SuspiciousUpdates_processed'
tree_ids = (
        [row[0] for row in arcpy.da.SearchCursor(
            fc, ['tree_uid'])])

In [2]:
tree_ids

[u'{02ae2d1f-0a9f-42ec-99d0-185d7e06ac78}']